<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/803/v2-lstm-600epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy pandas tensorflow

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, classification_report

# Load the data
data = pd.read_csv('MFCC_with_Output.csv')

data['MFCC'] = data['MFCC'].apply(lambda x: np.array([float(num) for num in x.strip('[]').split(',')]))

# Preparing the dataset
X = np.array(data['MFCC'].tolist())
y = data['Output'].values

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encode the labels
y_encoded = to_categorical(y)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Building the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(1, X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(y_encoded.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [3]:
# Train the model
model.fit(X_train, y_train, epochs=600, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/600
102/102 [==============================] - 21s 58ms/step - loss: 2.2064 - accuracy: 0.3530 - val_loss: 2.0364 - val_accuracy: 0.4034
Epoch 2/600
102/102 [==============================] - 2s 18ms/step - loss: 1.8263 - accuracy: 0.4060 - val_loss: 1.7273 - val_accuracy: 0.4047
Epoch 3/600
102/102 [==============================] - 2s 17ms/step - loss: 1.6896 - accuracy: 0.4127 - val_loss: 1.6815 - val_accuracy: 0.4047
Epoch 4/600
102/102 [==============================] - 2s 16ms/step - loss: 1.6671 - accuracy: 0.4183 - val_loss: 1.6643 - val_accuracy: 0.4047
Epoch 5/600
102/102 [==============================] - 1s 7ms/step - loss: 1.6489 - accuracy: 0.4204 - val_loss: 1.6541 - val_accuracy: 0.3985
Epoch 6/600
102/102 [==============================] - 1s 7ms/step - loss: 1.6366 - accuracy: 0.4272 - val_loss: 1.6472 - val_accuracy: 0.4010
Epoch 7/600
102/102 [==============================] - 1s 7ms/step - loss: 1.6282 - accuracy: 0.4263 - val_loss: 1.6400 - val_accuracy: 0

In [4]:
# Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy: {scores[1]*100}%')

Accuracy: 43.665435910224915%


In [5]:
# Predict classes
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calculate F1 score for each class
f1_scores = f1_score(y_true_classes, y_pred_classes, average=None)  # Returns an array of F1 scores for each class

# Print F1 scores for each class
for idx, f1 in enumerate(f1_scores):
    print(f'F1 Score for Class {idx}: {f1:.4f}')

# Optionally, print a full classification report
print(classification_report(y_true_classes, y_pred_classes))

26/26 [==============================] - 1s 3ms/step
F1 Score for Class 0: 0.0000
F1 Score for Class 1: 0.1967
F1 Score for Class 2: 0.1562
F1 Score for Class 3: 0.2676
F1 Score for Class 4: 0.0000
F1 Score for Class 5: 0.0000
F1 Score for Class 6: 0.3857
F1 Score for Class 7: 0.6132
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           2       0.26      0.16      0.20        75
           3       0.26      0.11      0.16        90
           4       0.47      0.19      0.27       102
           5       0.00      0.00      0.00        72
           6       0.00      0.00      0.00         9
           7       0.35      0.43      0.39       126
           8       0.50      0.79      0.61       329

    accuracy                           0.44       813
   macro avg       0.23      0.21      0.20       813
weighted avg       0.37      0.44      0.38       813

